In [14]:
import pandas as pd
import os
import json
import pickle
from surprise import Dataset, Reader


In [15]:
script_dir = os.path.dirname(os.path.abspath(__file__))
project_root = os.path.dirname(os.path.dirname(script_dir))

print(script_dir)

NameError: name '__file__' is not defined

In [30]:
# Chargement des datasets
def read_ratings(ratings_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/data/raw") -> pd.DataFrame:
    """Reads the CSV file containing movie ratings."""
    data = pd.read_csv(os.path.join(data_dir, ratings_csv))
    print("Dataset ratings loaded")
    return data

def read_movies(movies_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/data/raw") -> pd.DataFrame:
    """Reads the CSV file containing movie information."""
    df = pd.read_csv(os.path.join(data_dir, movies_csv))
    print("Dataset movies loaded")
    return df

def read_links(links_csv: str, data_dir: str = "/home/antoine/Ml_Ops_Movies_Reco/src/data/raw") -> pd.DataFrame:
    """Reads the CSV file containing movie information."""
    df = pd.read_csv(os.path.join(data_dir, links_csv))
    print("Dataset links loaded")
    return df


# Chargement du dernier modèle
def load_model(directory = "/home/antoine/Ml_Ops_Movies_Reco/data/model") :
    """Charge le modèle à partir d'un répertoire."""
    # Vérifier si le répertoire existe
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Le répertoire {directory} n'existe pas.")
    # Charger le modèle
    filepath = os.path.join(directory, 'model_svd.pkl')
    with open(filepath, 'rb') as file:
        model = pickle.load(file)
        print(f'Modèle chargé depuis {filepath}')
    return model

# Fonction pour obtenir des recommandations
def get_recommendations(user_id: int, model: SVD, ratings_df: pd.DataFrame, n_recommendations: int = 10):
    """Obtenir des recommandations pour un utilisateur donné."""
    # Créer un DataFrame contenant tous les films
    all_movies = ratings_df['movieId'].unique()

    # Obtenir les films déjà évalués par l'utilisateur
    rated_movies = ratings_df[ratings_df['userId'] == user_id]['movieId'].tolist()

    # Trouver les films non évalués par l'utilisateur
    unseen_movies = [movie for movie in all_movies if movie not in rated_movies]

    # Préparer les prédictions pour les films non évalués
    predictions = []
    for movie_id in unseen_movies:
        pred = model.predict(user_id, movie_id)
        predictions.append((movie_id, pred.est))  # Ajouter l'ID du film et la note prédite

    # Trier les prédictions par note prédite (descendant) et prendre les meilleures n_recommendations
    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n_recommendations]

    return top_n  # Retourner les meilleures recommandations

In [32]:
print("DEBUT DES CHARGEMENTS")
# Chargement de nos dataframe depuis mongo_db
ratings = read_ratings('ratings.csv')
movies = read_movies('movies.csv')
links = read_links('links.csv')

DEBUT DES CHARGEMENTS
Dataset ratings loaded
Dataset movies loaded
Dataset links loaded


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [ ]:
def bayesienne_mean(df, M, C):
    '''
    𝑀  = moyenne brute des notes des livres.
    𝑛  = la quantité totale de notes.
    𝑆  = la somme des notes.
    𝐶  = moyenne de la quantité de notes.
    '''
    moy_ba = (C*M+df.sum())/(C+df.count())
    return moy_ba

### PREPROCESSING RATINGS - UTILISATION MOYENNE BAYESIENNE

'''La moyenne bayésienne permet de réduire l'impact des évaluations extrêmes (très hautes ou très basses) en ramenant les évaluations vers une moyenne globale, ce qui peut être particulièrement utile si certains utilisateurs ont tendance à donner des notes très élevées ou très basses'''

def preprocessing_ratings(ratings_csv: str, load_data_dir) -> pd.DataFrame:
    """Reads the CSV file containing movie ratings."""
    df = pd.read_csv(os.path.join(load_data_dir, ratings_csv))
    print("Dataset ratings loaded")
    print("Calcul de la moyenne Bayesienne pour les notes")

    # quantité de notes par chaque film ainsi que la note moyenne par film
    movies_stats = df.groupby('movieId').agg({'rating': ('count', 'mean')})
    movies_stats.columns = ['count', 'mean']

    # moyenne de la quantité de notes.
    C = movies_stats['count'].mean()
    # moyenne brute des notes des livres.
    M = movies_stats['mean'].mean()

    # moyenne bayesienne par livre
    bay_moy_movie = df.groupby('movieId')['rating'].agg(
        lambda x: bayesienne_mean(x, M, C)  # Appeler bayesienne_mean avec les bonnes valeurs
    ).reset_index()

    # ajoutez une colonne à votre variable livre_stats
    movies_stats = pd.merge(left=movies_stats,
                            right=bay_moy_movie,
                            left_on='movieId',
                            right_on='movieId',
                            how='inner')

    return movies_stats

In [ ]:
import pandas as pd
import numpy as np

def bayesienne_mean(df, M, C):
    '''
    𝑀  = moyenne brute des notes des films.
    𝐶  = moyenne de la quantité de notes.
    '''
    moy_ba = (C * M + df.sum()) / (C + df.count())
    return moy_ba

def preprocessing_ratings(ratings_csv: str, load_data_dir) -> pd.DataFrame:
    """Reads the CSV file containing movie ratings and applies Bayesian mean."""
    df = pd.read_csv(os.path.join(load_data_dir, ratings_csv))
    print("Dataset ratings loaded")

    # Quantité de notes par chaque film ainsi que la note moyenne par film
    movies_stats = df.groupby('movieId').agg({'rating': ['count', 'mean']})
    movies_stats.columns = ['count', 'mean']

    # Moyenne de la quantité de notes.
    C = movies_stats['count'].mean()

    # Moyenne brute des notes des films.
    M = movies_stats['mean'].mean()

    # Calculer la moyenne bayésienne par film
    movies_stats['bayesian_mean'] = movies_stats.apply(lambda x: bayesienne_mean(df[df['movieId'] == x.name]['rating'], M, C), axis=1)

    # Remplacer les évaluations originales par les moyennes bayésiennes
    df['rating'] = df['movieId'].apply(lambda x: movies_stats.loc[x, 'bayesian_mean'])

    return df

# Exemple d'utilisation
if __name__ == "__main__":
    load_data_dir = "/opt/airflow/data/raw"  # Chemin vers le dossier contenant le CSV
    ratings_df = preprocessing_ratings('ratings.csv', load_data_dir)
    print(ratings_df.head())  # Afficher les premières lignes du DataFrame modifié

In [11]:

load_data_dir = "/home/antoine/Ml_Ops_Movies_Reco/data/raw/processed_ratings.csv"  # Chemin vers le dossier contenant le CSV
ratings_df = pd.read_csv(load_data_dir)
ratings_df.head()  # Afficher les premières lignes du DataFrame modifié

,userId,movieId,rating,timestamp
0,1,2,3.209414,1112486027
1,1,29,3.886141,1112484676
2,1,32,3.885546,1112484819
3,1,47,4.037850,1112484727
4,1,50,4.315561,1112484580


In [13]:
# Convert the 'timestamp' column from Unix timestamp to datetime
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')

# Display the updated DataFrame
print(ratings_df)

          userId  movieId    rating           timestamp
0              1        2  3.209414 2005-04-02 23:53:47
1              1       29  3.886141 2005-04-02 23:31:16
2              1       32  3.885546 2005-04-02 23:33:39
3              1       47  4.037850 2005-04-02 23:32:07
4              1       50  4.315561 2005-04-02 23:29:40
...          ...      ...       ...                 ...
20000258  138493    68954  3.970662 2009-11-13 15:42:00
20000259  138493    69526  2.990982 2009-12-03 18:31:48
20000260  138493    69644  3.277161 2009-12-07 18:10:57
20000261  138493    70286  3.809554 2009-11-13 15:42:24
20000262  138493    71619  3.203813 2009-10-17 20:25:36

[20000263 rows x 4 columns]


In [33]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [34]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  27278 non-null  int64  
 1   imdbId   27278 non-null  int64  
 2   tmdbId   27026 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 639.5 KB


In [11]:
movies.tail()

,movieId,title,genres,year
27286,131262,Piece by Piece,"['Computer Animation', 'Jukebox Musical', 'Pop...",2024.0
27287,131262,Transformers One,"['Action Epic', 'Adventure Epic', 'Computer An...",2024.0
27288,131262,Here,['Drama'],2024.0
27289,131262,Bhool Bhulaiyaa 3,"['Comedy', 'Horror']",2024.0
27290,131262,Singham Again,"['Action', 'Drama']",2024.0


In [35]:
def preprocessing_links(links_file):
    '''
    Chargement du dataset et modification type TmdbId en int'''
    df = pd.read_csv(links_file)
    print("Dataset links chargé")
    print('Modification du type de la colonne tmdbId en int')
    df['tmdbId'] = df.tmdbId.fillna(0)
    df['tmdbId'] = df.tmdbId.astype(int)
    # Définir le chemin vers le sous-dossier 'raw' dans le dossier parent 'data'
    output_dir = os.path.join("..", "data", "raw")  # ".." fait référence au dossier parent
    output_file = os.path.join(output_dir, "processed_links.csv")

    # Créer le dossier 'raw' s'il n'existe pas
    os.makedirs(output_dir, exist_ok=True)

    # Enregistrer le DataFrame en tant que fichier CSV
    try:
        df.to_csv(output_file, index=False)  # Enregistrer sans l'index
        print(f"Fichier enregistré avec succès sous {output_file}.")
    except Exception as e:
        print(f"Une erreur s'est produite lors de l'enregistrement du fichier : {e}")
    return df


In [36]:
data_dir = os.path.join("..", "src", "data", "raw")  # Chemin relatif vers le dossier
links_file = os.path.join(data_dir, "links.csv")
links_df = preprocessing_links(links_file)

Dataset links chargé
Modification du type de la colonne tmdbId en int
Fichier enregistré avec succès sous ../data/raw/processed_links.csv.


In [3]:
links2 = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/data/raw/links2.csv')

link = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/data/raw/processed_links.csv')

In [4]:
link.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [5]:
links2.head()

,Unnamed: 0,movieId,imdbId,tmdbId,cover_link
0,0,1,114709,862.0,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,1,2,113497,8844.0,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,2,3,113228,15602.0,https://m.media-amazon.com/images/M/MV5BMDkwYT...
3,3,4,114885,31357.0,https://m.media-amazon.com/images/M/MV5BZWU4Nz...
4,4,5,113041,11862.0,https://m.media-amazon.com/images/M/MV5BOTMwNz...


In [6]:
new_df = link.merge(links2[['cover_link', 'movieId']], left_on='movieId', right_on='movieId', how='left')

In [7]:
new_df.head()

,movieId,imdbId,tmdbId,cover_link
0,1,114709,862,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,113497,8844,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,113228,15602,https://m.media-amazon.com/images/M/MV5BMDkwYT...
3,4,114885,31357,https://m.media-amazon.com/images/M/MV5BZWU4Nz...
4,5,113041,11862,https://m.media-amazon.com/images/M/MV5BOTMwNz...


In [8]:
output_dir = os.path.join("..", "data", "raw")  # ".." fait référence au dossier parent
output_file = os.path.join(output_dir, "processed_links.csv")

# Créer le dossier 'raw' s'il n'existe pas
os.makedirs(output_dir, exist_ok=True)

# Enregistrer le DataFrame en tant que fichier CSV
try:
    new_df.to_csv(output_file, index=False)  # Enregistrer sans l'index
    print(f"Fichier enregistré avec succès sous {output_file}.")
except Exception as e:
    print(f"Une erreur s'est produite lors de l'enregistrement du fichier : {e}")


Fichier enregistré avec succès sous ../data/raw/processed_links.csv.


In [9]:
link = pd.read_csv('/home/antoine/Ml_Ops_Movies_Reco/data/raw/processed_links.csv')
link.head()

,movieId,imdbId,tmdbId,cover_link
0,1,114709,862,https://m.media-amazon.com/images/M/MV5BMDU2ZW...
1,2,113497,8844,https://m.media-amazon.com/images/M/MV5BZTk2Zm...
2,3,113228,15602,https://m.media-amazon.com/images/M/MV5BMDkwYT...
3,4,114885,31357,https://m.media-amazon.com/images/M/MV5BZWU4Nz...
4,5,113041,11862,https://m.media-amazon.com/images/M/MV5BOTMwNz...
